In [1]:
from azure.cosmos import exceptions, CosmosClient, PartitionKey
from dotenv import load_dotenv
import os
import json
from azure.identity import ClientSecretCredential, DefaultAzureCredential

In [2]:
load_dotenv(dotenv_path='./.env')

True

#### Use default CosmosKey as way to authenticate

In [3]:
client = CosmosClient(os.getenv('CosmosEndpoint'), os.getenv('CosmosKey'))

In [4]:
database = client.get_database_client(os.getenv('CosmosDatabase'))

In [5]:
container = database.get_container_client(os.getenv('CosmosContainer'))

In [6]:
queryText = "SELECT * FROM c WHERE c.sessionId = @sessionId"
results = container.query_items(
    query=queryText,
    parameters=[
        dict(
            name="@sessionId",
            value="nlp28ve47vgkqbcv4pcnmp",
        )
    ],
    enable_cross_partition_query=False,
)

In [7]:
items = [item for item in results]
output = json.dumps(items, indent=True)
output

'[\n {\n  "id": "wh5cjidjuzgp1cevkvlbd",\n  "type": "Session",\n  "sessionId": "nlp28ve47vgkqbcv4pcnmp",\n  "name": "basicstuff",\n  "chainType": "stuff",\n  "feature": "chat",\n  "indexId": "eea23eb8a76a4270a2cc1c951b0f587f",\n  "indexType": "cogsearchvs",\n  "indexName": "Bumble",\n  "llmModel": "gpt3.5",\n  "timestamp": "1691773949041",\n  "tokenUsed": 0,\n  "embeddingModelType": "azureopenai",\n  "_rid": "7owtAIkXtMtNAAAAAAAAAA==",\n  "_self": "dbs/7owtAA==/colls/7owtAIkXtMs=/docs/7owtAIkXtMtNAAAAAAAAAA==/",\n  "_etag": "\\"0300a382-0000-0500-0000-64d66c350000\\"",\n  "_attachments": "attachments/",\n  "_ts": 1691774005\n },\n {\n  "id": "a1f6ae3e-c6d3-44d5-80ec-4c0f0d337a0f",\n  "type": "Message",\n  "role": "User",\n  "sessionId": "nlp28ve47vgkqbcv4pcnmp",\n  "tokens": 0,\n  "timestamp": "2023-08-11T17:12:32.991029",\n  "content": "What is this document about?",\n  "_rid": "7owtAIkXtMtOAAAAAAAAAA==",\n  "_self": "dbs/7owtAA==/colls/7owtAIkXtMs=/docs/7owtAIkXtMtOAAAAAAAAAA==/",\n 

#### Use SPN to authenticate

#### Get the RoleDefinitionId
az cosmosdb sql role definition list --account-name dataaichatgpt --resource-group dataai

### Add SPN to default Reader role
New-AzCosmosDBSqlRoleAssignment -AccountName "dataaichatgpt" -ResourceGroupName "dataai" -RoleDefinitionId "/subscriptions/e2171f6d-2650-45e6-af7e-6d6e44ca92b1/resourceGroups/dataai/providers/Microsoft.DocumentDB/databaseAccounts/dataaichatgpt/sqlRoleDefinitions/00000000-0000-0000-0000-000000000002" -Scope "/" -PrincipalId "28c9d8ed-b960-4050-a600-8d5159470207"

In [13]:
aadCredentials = ClientSecretCredential(
        tenant_id=os.getenv('TenantId'),
        client_id=os.getenv('ClientId'),
        client_secret=os.getenv('ClientSecret'))

In [14]:
aadClient = CosmosClient(os.getenv('CosmosEndpoint'), aadCredentials)

In [15]:
aadDb = aadClient.get_database_client(os.getenv('CosmosDatabase'))
aadContainer = aadDb.get_container_client(os.getenv('CosmosContainer'))

In [16]:
print("Container info: " + str(aadContainer.read()))

Container info: {'id': 'chatgpt', 'indexingPolicy': {'indexingMode': 'consistent', 'automatic': True, 'includedPaths': [{'path': '/*'}], 'excludedPaths': [{'path': '/"_etag"/?'}]}, 'partitionKey': {'paths': ['/sessionId'], 'kind': 'Hash', 'version': 2}, 'uniqueKeyPolicy': {'uniqueKeys': []}, 'conflictResolutionPolicy': {'mode': 'LastWriterWins', 'conflictResolutionPath': '/_ts', 'conflictResolutionProcedure': ''}, 'geospatialConfig': {'type': 'Geography'}, '_rid': '7owtAIkXtMs=', '_ts': 1691772366, '_self': 'dbs/7owtAA==/colls/7owtAIkXtMs=/', '_etag': '"00000a00-0000-0500-0000-64d665ce0000"', '_docs': 'docs/', '_sprocs': 'sprocs/', '_triggers': 'triggers/', '_udfs': 'udfs/', '_conflicts': 'conflicts/'}


In [17]:
queryText = "SELECT * FROM c WHERE c.sessionId = @sessionId"
results = container.query_items(
    query=queryText,
    parameters=[
        dict(
            name="@sessionId",
            value="nlp28ve47vgkqbcv4pcnmp",
        )
    ],
    enable_cross_partition_query=False,
)

In [18]:
items = [item for item in results]
output = json.dumps(items, indent=True)
output

'[\n {\n  "id": "wh5cjidjuzgp1cevkvlbd",\n  "type": "Session",\n  "sessionId": "nlp28ve47vgkqbcv4pcnmp",\n  "name": "basicstuff",\n  "chainType": "stuff",\n  "feature": "chat",\n  "indexId": "eea23eb8a76a4270a2cc1c951b0f587f",\n  "indexType": "cogsearchvs",\n  "indexName": "Bumble",\n  "llmModel": "gpt3.5",\n  "timestamp": "1691773949041",\n  "tokenUsed": 0,\n  "embeddingModelType": "azureopenai",\n  "_rid": "7owtAIkXtMtNAAAAAAAAAA==",\n  "_self": "dbs/7owtAA==/colls/7owtAIkXtMs=/docs/7owtAIkXtMtNAAAAAAAAAA==/",\n  "_etag": "\\"0300a382-0000-0500-0000-64d66c350000\\"",\n  "_attachments": "attachments/",\n  "_ts": 1691774005\n },\n {\n  "id": "a1f6ae3e-c6d3-44d5-80ec-4c0f0d337a0f",\n  "type": "Message",\n  "role": "User",\n  "sessionId": "nlp28ve47vgkqbcv4pcnmp",\n  "tokens": 0,\n  "timestamp": "2023-08-11T17:12:32.991029",\n  "content": "What is this document about?",\n  "_rid": "7owtAIkXtMtOAAAAAAAAAA==",\n  "_self": "dbs/7owtAA==/colls/7owtAIkXtMs=/docs/7owtAIkXtMtOAAAAAAAAAA==/",\n 